In [15]:
import pandas as pd

df = pd.read_parquet("data/")

In [18]:
df.loc[4]

brand                                                            Iveco
model                                                 DAILY 35S16HA8 V
color                                                            Biały
seats                                                                7
year                                                              2025
fuel                                                            Diesel
capacity                                                     2 287 cm3
power                                                           160 KM
body_type                                             Furgon (blaszak)
gearbox                                                   Automatyczna
mileage                                                           5 km
condition                                                         Nowy
accident_free                                                      Tak
country_of_origin                                               Polska
title 